In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd
from bs4 import BeautifulSoup
import datetime
import time
import os
import csv
import traceback

def aggregate_data(header1, data1, header2, data2, shift):
    shift = 4 if shift < 2 else 3
    return header1 + header2[shift:], [data1[i] + data2[i][shift:] for i in range(len(data1))]

def get_batch_urls(year: str):
    base = "https://www.mlb.com/stats"
    player_hitting = f"{base}/{year}"
    player_pitching = f"{base}/pitching/{year}?sortState=asc"
    team_hitting = f"{base}/team/{year}"
    team_pitching = f"{base}/team/pitching/{year}?sortState=asc"
    return player_hitting, player_pitching, team_hitting, team_pitching

def scrape_table_data(driver, idx):
    data = []
    while True:
        # Wait for the table to load
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "table.bui-table")))
        
        soup = BeautifulSoup(driver.page_source, 'lxml')
        
        table = soup.find('table', class_='bui-table')
        headers = table.thead.find_all('abbr')
        header = ["index"]
        for i in range(len(headers)):
            if i % 2 == 0:
                header.append(headers[i].text)
            if i == 0 and idx < 2:
                header.append("position")



        rows = table.tbody.find_all('tr')
        
        for row in rows:
            th = row.find('th')
            index = th.select('div[class^=index-]')[0].text
            if (len(th.div.select('[class^=full-]'))) == 2:
                name = th.div.select('[class^=full-]')[0].text + " " + th.div.select('[class^=full-]')[1].text
            else:
                name = th.div.select('[class^=full-]')[0].text
            if idx < 2:
                position = th.div.select_one('div[class^=position-]').text
                row_data = [index, name, position]
            else:
                row_data = [index, name]
            cells = row.find_all('td')
            row_data = row_data + [cell.text.strip() for cell in cells]
            data.append(row_data)
        
        try:
            # Find the Next button and click it
            next_button = driver.find_element(By.CSS_SELECTOR, "button[aria-label='next page button']")
            if next_button.is_enabled():
                next_button.click()
                time.sleep(3)
            else:
                break
        except NoSuchElementException:
            break

    return header, data


def parse_year_data(year):
    options = Options()
    # options.add_argument("--headless")  # Enable headless mode for background processing
    driver = webdriver.Chrome(options=options)

    driver.get("https://www.mlb.com/stats/2021")
    current_time = datetime.datetime.now().isoformat() + 'Z' 

    # Create a cookie
    cookie = {
        'name': 'OptanonAlertBoxClosed',
        'value': current_time,
        'domain': '.mlb.com',  # Set this to the domain for which you want the cookie
        'path': '/',
    }
    driver.add_cookie(cookie)

    year = str(year)
    urls = get_batch_urls(year)
    terms = ["player_hitting", "player_pitching", "team_hitting", "team_pitching"]
    folder = f"./mlb/{year}"
    if not os.path.exists(folder):
        os.makedirs(folder)

    for i in range(4):
        driver.get(urls[i])
        header, data = scrape_table_data(driver,i)
        try:
            # Find the Next button and click it
            driver.get(urls[i])
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//*[@id='stats-app-root']/section/section/div[1]/div[2]/div/div[1]/div/div[2]/button")))
            next_button = driver.find_element(By.XPATH, "//*[@id='stats-app-root']/section/section/div[1]/div[2]/div/div[1]/div/div[2]/button")
            if next_button.is_enabled():
                next_button.click()
                time.sleep(3)
                
            else:
                break
        except:
            traceback.print_exc()
            break
        header1, data1 = scrape_table_data(driver,i)
        print(len(header), len(header1))
        print(len(data), len(data1))
        print(header)
        print(header1)
        print(data)
        print(data1)
        header, data = aggregate_data(header, data, header1, data1,i)


        filename = f"{folder}/{terms[i]}.csv"

        with open(filename, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(header)
            writer.writerows(data)

    driver.quit()  # Don't forget to close the driver

def main():
    year = 2007
    while year <= 2023:
        try:
            parse_year_data(year)  
        except:
            print(f"Failed to scrape data for {year}")
            continue
        year += 1

if __name__ == "__main__":
    main()


Failed to scrape data for 2007
Failed to scrape data for 2007
Failed to scrape data for 2007
Failed to scrape data for 2007
Failed to scrape data for 2007


## Visualize

In [ ]:
import polars as pl

df = pl.read_csv("player_hitting_2003.csv")
df

index,PLAYER,position,TEAM,G,AB,R,H,2B,3B,HR,RBI,BB,SO,SB,CS,AVG,OBP,SLG,OPS
i64,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64
1,"""Barry Bonds""","""LF""","""SF""",130,390,111,133,22,1,45,90,148,58,7,0,0.341,0.529,0.749,1.278
2,"""Albert Pujols""","""LF""","""STL""",157,591,137,212,51,1,43,124,79,65,5,1,0.359,0.439,0.667,1.106
3,"""Todd Helton""","""1B""","""COL""",160,583,135,209,49,5,33,117,111,72,0,4,0.358,0.458,0.63,1.088
4,"""Gary Sheffield…","""RF""","""ATL""",155,576,126,190,37,2,39,132,86,55,18,4,0.33,0.419,0.604,1.023
5,"""Carlos Delgado…","""1B""","""TOR""",161,570,117,172,38,1,42,145,109,137,0,0,0.302,0.426,0.593,1.019
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
161,"""José Hernández…","""3B""","""PIT""",150,519,58,117,18,3,13,57,46,177,2,1,0.225,0.287,0.347,0.634
163,"""Alex Cora""","""2B""","""LA""",148,477,39,119,24,3,4,34,16,59,4,2,0.249,0.287,0.338,0.625
164,"""Cesar Izturis""","""SS""","""LA""",158,558,47,140,21,6,1,40,25,70,10,5,0.251,0.282,0.315,0.597


In [ ]:
df = pl.read_csv("player_pitching_2003.csv")
df

index,PLAYER,position,TEAM,W,L,ERA,G,GS,CG,SHO,SV,SVO,IP,H,R,ER,HR,HB,BB,SO,WHIP,AVG
i64,str,str,str,i64,i64,f64,i64,i64,i64,i64,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,f64,f64
1,"""Pedro Martinez…","""P""","""BOS""",14,4,2.22,29,29,3,0,0,0,186.2,147,52,46,7,9,47,206,1.04,0.215
2,"""Jason Schmidt""","""P""","""SF""",17,5,2.34,29,29,5,3,0,0,207.2,152,56,54,14,5,46,208,0.95,0.2
3,"""Kevin Brown""","""P""","""LA""",14,9,2.39,32,32,0,0,0,0,211.0,184,67,56,11,5,56,185,1.14,0.236
4,"""Mark Prior""","""P""","""CHC""",18,6,2.43,30,30,3,1,0,0,211.1,183,67,57,15,9,50,245,1.1,0.231
5,"""Tim Hudson""","""P""","""OAK""",16,7,2.7,34,34,3,2,0,0,240.0,197,84,72,15,10,61,162,1.08,0.223
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
88,"""Shane Reynolds…","""P""","""ATL""",11,9,5.43,30,29,0,0,0,0,167.1,191,104,101,20,8,59,94,1.49,0.293
89,"""Wayne Franklin…","""P""","""MIL""",10,13,5.5,36,34,1,1,0,0,194.2,201,129,119,36,10,94,116,1.52,0.268
90,"""Jeremy Bonderm…","""P""","""DET""",6,19,5.56,33,28,0,0,0,0,162.0,193,118,100,23,4,58,108,1.55,0.294


In [ ]:
df = pl.read_csv("team_pitching_2003.csv")
df

index,TEAM,LEAGUE,W,L,ERA,G,GS,CG,SHO,SV,SVO,IP,H,R,ER,HR,HB,BB,SO,WHIP,AVG
i64,str,str,i64,i64,f64,i64,i64,i64,i64,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,f64,f64
1,"""Los Angeles Do…","""NL""",85,77,3.16,162,162,3,17,58,66,1457.2,1254,556,511,127,40,526,1289,1.22,0.234
2,"""Oakland Athlet…","""AL""",96,66,3.63,162,162,16,14,48,60,1441.2,1336,643,582,140,54,499,1018,1.27,0.246
3,"""San Francisco …","""NL""",100,61,3.73,161,161,7,10,43,60,1437.1,1349,638,595,136,43,546,1006,1.32,0.25
4,"""Seattle Marine…","""AL""",93,69,3.76,162,162,8,15,38,52,1441.0,1340,637,602,173,54,466,1001,1.25,0.247
5,"""Chicago Cubs""","""NL""",88,74,3.83,162,162,13,14,36,51,1456.1,1304,683,619,143,71,617,1404,1.32,0.241
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
26,"""Kansas City Ro…","""AL""",83,79,5.05,162,162,7,10,36,64,1438.2,1569,867,808,190,66,566,865,1.48,0.279
27,"""Cincinnati Red…","""NL""",69,93,5.09,162,162,4,5,38,64,1446.1,1578,886,818,209,48,590,932,1.5,0.278
28,"""Colorado Rocki…","""NL""",74,88,5.2,162,162,3,4,34,54,1420.0,1629,892,821,200,84,552,866,1.54,0.29


In [ ]:
df = pl.read_csv("team_hitting_2003.csv")
df

index,TEAM,LEAGUE,G,AB,R,H,2B,3B,HR,RBI,BB,SO,SB,CS,AVG,OBP,SLG,OPS
i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64
1,"""Texas Rangers""","""AL""",162,5664,826,1506,274,36,239,799,488,1052,65,25,0.266,0.33,0.454,0.784
2,"""Boston Red Sox…","""AL""",162,5769,961,1667,371,40,238,932,620,943,88,35,0.289,0.36,0.491,0.851
3,"""Atlanta Braves…","""NL""",162,5670,907,1608,321,31,235,872,545,933,68,22,0.284,0.349,0.475,0.824
4,"""New York Yanke…","""AL""",163,5605,877,1518,304,14,230,845,684,1042,98,33,0.271,0.356,0.453,0.809
5,"""Chicago White …","""AL""",162,5487,791,1445,303,19,220,766,519,916,77,29,0.263,0.331,0.446,0.777
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
26,"""Seattle Marine…","""AL""",162,5561,795,1509,290,33,139,759,586,989,108,37,0.271,0.344,0.41,0.754
27,"""Tampa Bay Devi…","""AL""",162,5654,715,1501,298,38,137,678,420,1030,142,42,0.265,0.32,0.404,0.724
28,"""San Diego Padr…","""NL""",162,5531,678,1442,257,32,128,641,565,1073,76,39,0.261,0.333,0.388,0.721
